In [1]:
import random
import numpy as np
import os
import pandas as pd
import torch
import pyarrow
from pysurvival.models.multi_task import NeuralMultiTaskModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import RepeatedKFold, train_test_split
import matplotlib

matplotlib.use('Agg')  # Use Agg backend
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set up plotting style for scientific publication
plt.style.use('seaborn-whitegrid')
sns.set_context("paper", font_scale=1.5)
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']


def seed_everything(seed=42):
    """Set random seed for reproducibility."""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    np.seterr(divide='ignore', invalid='ignore')


SEED = 12121
print(f"Random seed: {SEED}")
seed_everything(SEED)

# Read data
dataset = pd.read_csv('/Users/wangyuanzhang/Desktop/LHCC/r/corrdata.csv')
dataset.columns = [name.replace('.', ' ') for name in dataset.columns]
time_column = 'Survival_months'
event_column = 'Status'
features = np.setdiff1d(dataset.columns, [time_column, event_column]).tolist()

print(f"Dataset shape: {dataset.shape}")

# Split dataset
index_train, index_test = train_test_split(range(dataset.shape[0]), test_size=0.3, random_state=SEED)
data_train = dataset.loc[index_train].reset_index(drop=True)
data_test = dataset.loc[index_test].reset_index(drop=True)

data_train.to_csv('/Users/wangyuanzhang/Desktop/LHCC/r/train_data.csv', index=False)
data_test.to_csv('/Users/wangyuanzhang/Desktop/LHCC/r/test_data.csv', index=False)

# Prepare inputs
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train[time_column], data_test[time_column]
E_train, E_test = data_train[event_column], data_test[event_column]

print(f"Training set shape: {data_train.shape}, Test set shape: {data_test.shape}")


class BaseFun:
    def __init__(self):
        self.model = None
        self.tuning_result = None
        self.space = None

    @staticmethod
    def parse_param(param):
        return ','.join([f"{key} = {repr(value)}" for key, value in param.items()])

    @staticmethod
    def get_random_param(space):
        param = {}
        for key, value in space.items():
            if key == 'structure':
                param[key] = [
                    {
                        'activation': random.choice(value['activations']),
                        'num_units': random.choice(value['num_units'])
                    }
                    for _ in range(random.choice(value['num_layers']))
                ]
            else:
                param[key] = random.choice(value)
        return param

    @staticmethod
    def fit_model(x_train, t_train, e_train, **param):
        model = NeuralMultiTaskModel(**param)
        model.fit(x_train, t_train, e_train, verbose=False)
        return model

    def tuning_and_construct(self, X, T, E, max_iter=100):
        self.tuning_result = self.tuning_with_space(X, T, E, self.space, max_iter=max_iter)
        self.model = self.fit_model(X, T, E, **self.tuning_result['best_param'])
        return self.model

    def tuning_with_space(self, x, t, e, space, max_iter=100):
        x, t, e = map(np.array, [x, t, e])
        scores = []
        best_score, best_param = 0, {}

        for num in range(1, max_iter + 1):
            param = self.get_random_param(space)
            print(f'Iteration {num}', end=' ... ')

            rkf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=SEED)
            score_iter = []

            for train_index, test_index in rkf.split(x):
                x_train, x_test = x[train_index], x[test_index]
                t_train, t_test = t[train_index], t[test_index]
                e_train, e_test = e[train_index], e[test_index]

                try:
                    model = self.fit_model(x_train, t_train, e_train, **param)
                    score = concordance_index(model, x_test, t_test, e_test)
                    score_iter.append(score)
                except Exception as err:
                    print(f"Error: {str(err)}")
                    continue

            mean_score = np.mean(score_iter)
            print(f'Average c-index: {mean_score:.4f}', end=' ')

            if mean_score > best_score:
                best_score, best_param = mean_score, param
                print('Best so far')
            else:
                print('')

            scores.append({'iteration': num, 'mean_score': mean_score, 'param': param})

        return {'best_param': best_param, 'best_score': best_score, 'scores': scores}
# NMTLR
class NMTLR(BaseFun):
    def __init__(self):
        super().__init__()
        self.space = {
            'structure': {
                'num_layers': [1, 2, 3, 4, 5],
                'num_units': list(range(8, 101)),
                'activations': [
                    "Atan", "BentIdentity", "BipolarSigmoid", "CosReLU",
                    "Gaussian", "Hardtanh", "InverseSqrt", "LeakyReLU",
                    "LeCunTanh", "LogLog", "LogSigmoid", "ReLU", "SELU",
                    "Sigmoid", "Sinc", "SinReLU", "Softmax", "Softplus",
                    "Softsign", "Swish", "Tanh"
                ]
            },
            'optimizer': ['adadelta', 'adagrad', 'adam', 'adamax', 'rmsprop', 'sgd'],
            'lr': [round(1e-5 * i, 5) for i in range(1, 101)],
            'num_epochs': list(range(50, 1001)),
            'dropout': [round(0.1 * i, 2) for i in range(1, 5)],
            'l2_reg': [round(0.0001 * i, 5) for i in range(1, 101)],
            'l2_smooth': [round(0.0001 * i, 5) for i in range(1, 101)],
            'batch_normalization': [False, True]
        }

    def fit_model(self, X, T, E, **kwargs):
        structure = kwargs.pop('structure', [{'activation': 'ReLU', 'num_units': 128}])
        bins = kwargs.pop('bins', 100)
        self.model = NeuralMultiTaskModel(structure=structure, bins=bins)
        self.model.fit(X, T, E, **kwargs)
        return self.model

# DeepSurv
class DeepSurv(BaseFun):
    def __init__(self):
        super().__init__()
        self.space = {
            'structure': {
                'num_layers': [1, 2, 3, 4, 5],
                'num_units': list(range(8, 101)),
                'activations': [
                    "Atan", "BentIdentity", "BipolarSigmoid", "CosReLU",
                    "Gaussian", "Hardtanh", "InverseSqrt", "LeakyReLU",
                    "LeCunTanh", "LogLog", "LogSigmoid", "ReLU", "SELU",
                    "Sigmoid", "Sinc", "SinReLU", "Softmax", "Softplus",
                    "Softsign", "Swish", "Tanh"
                ]
            },
            'optimizer': ['adadelta', 'adagrad', 'adam', 'adamax', 'rmsprop', 'sgd'],
            'lr': [round(1e-5 * i, 5) for i in range(1, 101)],
            'num_epochs': list(range(50, 5001)),
            'dropout': [round(0.1 * i, 2) for i in range(1, 5)],
            'l2_reg': [round(0.0001 * i, 5) for i in range(1, 101)],
            'batch_normalization': [False, True]
        }

    def fit_model(self, X, T, E, **kwargs):
        structure = kwargs.pop('structure', [{'activation': 'ReLU', 'num_units': 128}])
        bins = kwargs.pop('bins', 100)
        self.model = NeuralMultiTaskModel(structure=structure, bins=bins)
        self.model.fit(X, T, E, **kwargs)
        return self.model

# RSF
from pysurvival.models.survival_forest import RandomSurvivalForestModel

class RSF(BaseFun):
    def __init__(self):
        super().__init__()
        self.space = {
            'num_trees': list(range(20, 1001)),
            'max_features': ['sqrt', 'log2', 'all', 0.1, 0.2],
            'min_node_size': list(range(5, 81)),
            'sample_size_pct': [round(0.2 * i, 2) for i in range(1, 5)],
            'importance_mode': ['impurity', 'impurity_corrected', 'permutation', 'normalized_permutation']
        }

    def fit_model(self, X, T, E, **kwargs):
        num_trees = kwargs.pop('num_trees', 100)
        self.model = RandomSurvivalForestModel(num_trees=num_trees)
        self.model.fit(X, T, E, seed=SEED, **kwargs)
        return self.model

# CPH
from pysurvival.models.semi_parametric import CoxPHModel

def cph(X_train, T_train, E_train):
    model = CoxPHModel()
    model.fit(X_train, T_train, E_train, lr=0.2, l2_reg=0.01)
    return model

# Handle missing values
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)

# Fit CPH model
cph_model = cph(X_train, T_train, E_train)
c_index_train = concordance_index(cph_model, X_train, T_train, E_train)
c_index_test = concordance_index(cph_model, X_test, T_test, E_test)
print(f'C-index of train: {c_index_train:.4f}; C-index of test: {c_index_test:.4f}')

from pysurvival.utils.display import integrated_brier_score

ibs = integrated_brier_score(cph_model, X_train, T_train, E_train, t_max=None, figure_size=(20, 6.5))
print(f'IBS: {ibs:.2f}')

# Model tuning and construction
max_iter = 1000

# NMTLR
nmtlr = NMTLR()
nmtlr.tuning_and_construct(X_train, T_train, E_train, max_iter=max_iter)
c_index_train = concordance_index(nmtlr.model, X_train, T_train, E_train)
c_index_test = concordance_index(nmtlr.model, X_test, T_test, E_test)
print(f'NMTLR - C-index of train: {c_index_train:.4f}; C-index of test: {c_index_test:.4f}')

ibs = integrated_brier_score(nmtlr.model, X_test, T_test, E_test, figure_size=(20, 6.5))
print(f'NMTLR - IBS: {ibs:.2f}')

# DeepSurv
deepsurv = DeepSurv()
deepsurv.tuning_and_construct(X_train, T_train, E_train, max_iter=max_iter)
c_index_train = concordance_index(deepsurv.model, X_train, T_train, E_train)
c_index_test = concordance_index(deepsurv.model, X_test, T_test, E_test)
print(f'DeepSurv - C-index of train: {c_index_train:.4f}; C-index of test: {c_index_test:.4f}')

ibs = integrated_brier_score(deepsurv.model, X_test, T_test, E_test, figure_size=(20, 6.5))
print(f'DeepSurv - IBS: {ibs:.2f}')

# RSF
rsf = RSF()
rsf.tuning_and_construct(X_train, T_train, E_train, max_iter=max_iter)
c_index_train = concordance_index(rsf.model, X_train, T_train, E_train)
c_index_test = concordance_index(rsf.model, X_test, T_test, E_test)
print(f'RSF - C-index of train: {c_index_train:.4f}; C-index of test: {c_index_test:.4f}')

ibs = integrated_brier_score(rsf.model, X_test, T_test, E_test, figure_size=(20, 6.5))
print(f'RSF - IBS: {ibs:.2f}')

# Save best parameters
with open('/Users/wangyuanzhang/Desktop/LHCC/python/best_parameters.txt', 'w') as f:
    f.write(f"NMTLR best parameters: {nmtlr.tuning_result['best_param']}\n")
    f.write(f"DeepSurv best parameters: {deepsurv.tuning_result['best_param']}\n")
    f.write(f"RSF best parameters: {rsf.tuning_result['best_param']}\n")


import os
import joblib
import numpy as np
import pandas as pd
from pysurvival.utils.metrics import brier_score
from sklearn.calibration import calibration_curve
from sklearn.metrics import roc_curve, auc

save_path = '/Users/wangyuanzhang/Desktop/LHCC/python'
os.makedirs(save_path, exist_ok=True)


# Helper function for saving plots
def save_plot(fig, filename):
    fig.savefig(os.path.join(save_path, filename), dpi=300, bbox_inches='tight')
    plt.close(fig)


# Brier Score and IBS calculation
def calculate_brier_and_ibs(model, X, T, E, t_max):
    times, brier_scores = brier_score(model, X, T, E, t_max=t_max, use_mean_point=True)
    ibs = np.trapz(brier_scores, times) / t_max
    return times, brier_scores, ibs


# Calculate Brier scores and IBS for all models
t_max = max(T_test)
results = {
    'NMTLR': calculate_brier_and_ibs(nmtlr.model, X_test, T_test, E_test, t_max),
    'DeepSurv': calculate_brier_and_ibs(deepsurv.model, X_test, T_test, E_test, t_max),
    'RSF': calculate_brier_and_ibs(rsf.model, X_test, T_test, E_test, t_max),
    'CPH': calculate_brier_and_ibs(cph_model, X_test, T_test, E_test, t_max)
}

# Plot IBS comparison
fig, ax = plt.subplots(figsize=(10, 6))
for model, (times, scores, ibs) in results.items():
    ax.plot(times, scores, label=f'{model} (IBS: {ibs:.4f})')
ax.set_xlabel('Time')
ax.set_ylabel('Brier Score')
ax.set_title('Integrated Brier Score Comparison')
ax.legend()
save_plot(fig, 'IBS_comparison.png')


# Generate predictions for different time points
def generate_prediction_by_year(time_, file_name):
    index = ~((T_test <= time_) & (E_test == 0))
    X_ = X_test.loc[index, :].reset_index(drop=True)
    T_ = T_test[index].reset_index(drop=True)
    E_ = E_test[index].reset_index(drop=True)

    predictions = {
        'NMTLR': nmtlr.model.predict_survival(X_, t=time_),
        'DeepSurv': deepsurv.model.predict_survival(X_, t=time_),
        'RSF': rsf.model.predict_survival(X_, t=time_),
        'CPH': cph_model.predict_survival(X_, t=time_)
    }

    df = pd.DataFrame({
        'event': [1 if T_[i] >= time_ else 0 for i in range(len(T_))] * 4,
        'survival': np.concatenate([pred for pred in predictions.values()]),
        'models': np.repeat(list(predictions.keys()), len(T_))
    })

    df.to_csv(os.path.join(save_path, file_name), index=False)
    return df


# Generate predictions for 1, 3, and 5 years
pred_1yr = generate_prediction_by_year(12, '1_year_prediction.csv')
pred_3yr = generate_prediction_by_year(36, '3_year_prediction.csv')
pred_5yr = generate_prediction_by_year(60, '5_year_prediction.csv')


# Plot ROC curves
def plot_roc_curve(predictions, title):
    fig, ax = plt.subplots(figsize=(10, 8))
    for model in predictions['models'].unique():
        fpr, tpr, _ = roc_curve(predictions[predictions['models'] == model]['event'],
                                predictions[predictions['models'] == model]['survival'])
        roc_auc = auc(fpr, tpr)
        ax.plot(fpr, tpr, label=f'{model} (AUC = {roc_auc:.3f})')

    ax.plot([0, 1], [0, 1], linestyle='--', color='gray')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'ROC Curves - {title}')
    ax.legend(loc='lower right')
    save_plot(fig, f'ROC_curves_{title}.png')


plot_roc_curve(pred_1yr, '1 Year')
plot_roc_curve(pred_3yr, '3 Years')
plot_roc_curve(pred_5yr, '5 Years')


# Plot calibration curves
def plot_calibration_curve(predictions, title):
    fig, ax = plt.subplots(figsize=(10, 8))
    for model in predictions['models'].unique():
        model_pred = predictions[predictions['models'] == model]
        prob_true, prob_pred = calibration_curve(model_pred['event'], model_pred['survival'], n_bins=10)
        ax.plot(prob_pred, prob_true, marker='o', label=model)

    ax.plot([0, 1], [0, 1], linestyle='--', color='gray')
    ax.set_xlabel('Mean Predicted Probability')
    ax.set_ylabel('Fraction of Positives')
    ax.set_title(f'Calibration Curves - {title}')
    ax.legend(loc='lower right')
    save_plot(fig, f'Calibration_curves_{title}.png')


plot_calibration_curve(pred_1yr, '1 Year')
plot_calibration_curve(pred_3yr, '3 Years')
plot_calibration_curve(pred_5yr, '5 Years')


# Save models
import pickle
from pysurvival.utils import save_model
class ModelWrapper:
    def __init__(self, model):
        self.model = model

    def predict_survival(self, X, times):
        return self.model.predict_survival(X, times)

# 保存模型
joblib.dump(ModelWrapper(nmtlr.model), os.path.join(save_path, 'NMTLR_model.joblib'))
joblib.dump(ModelWrapper(deepsurv.model), os.path.join(save_path, 'DeepSurv_model.joblib'))
save_model(rsf.model, os.path.join(save_path,'RSF.zip'))
joblib.dump(ModelWrapper(cph_model), os.path.join(save_path, 'CPH_model.joblib'))

print("All models and results have been saved.")


# Feature importance
def variable_importance(model, X, T, E):
    base_c_index = concordance_index(model, X, T, E)
    importances = []
    for column in X.columns:
        X_permuted = X.copy()
        X_permuted[column] = np.random.permutation(X_permuted[column])
        c_index_permuted = concordance_index(model, X_permuted, T, E)
        importances.append((base_c_index - c_index_permuted) / base_c_index)
    return importances


imp_df = pd.DataFrame(
    {
        'DeepSurv': variable_importance(deepsurv.model, X_test, T_test, E_test),
        'NMTLR': variable_importance(nmtlr.model, X_test, T_test, E_test),
        'RSF': variable_importance(rsf.model, X_test, T_test, E_test),
        'CPH': variable_importance(cph_model, X_test, T_test, E_test)
    },
    index=X_test.columns
)
imp_df['Average'] = imp_df.mean(axis=1)
imp_df = imp_df.sort_values(by='Average', ascending=False)

# Plot feature importance
fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(imp_df, cmap='YlOrRd', annot=True, fmt='.3f', cbar_kws={'label': 'Importance'})
ax.set_title('Feature Importance Across Models')
ax.set_xlabel('Models')
ax.set_ylabel('Features')
save_plot(fig, 'Feature_importance.png')

print("Feature importance analysis completed and saved.")

Duplicate key in file '/Users/wangyuanzhang/.matplotlib/matplotlibrc', line 2 ('backend: TkAgg')
<ipython-input-1-21e1ce268764>:18: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


Random seed: 12121
Dataset shape: (1536, 18)
Training set shape: (1075, 18), Test set shape: (461, 18)
Performing Newton-Raphson optimization
 * Iteration #1 - Loss = 4558.326 - ||grad||^2 = 1076.38576
 * Iteration #2 - Loss = 4379.863 - ||grad||^2 = 850.91230
 * Iteration #3 - Loss = 4304.302 - ||grad||^2 = 683.29373
 * Iteration #4 - Loss = 4260.752 - ||grad||^2 = 551.49899
 * Iteration #5 - Loss = 4232.968 - ||grad||^2 = 446.16844
 * Iteration #6 - Loss = 4214.352 - ||grad||^2 = 361.31936
 * Iteration #7 - Loss = 4201.535 - ||grad||^2 = 292.68903
 * Iteration #8 - Loss = 4192.569 - ||grad||^2 = 237.05407
 * Iteration #9 - Loss = 4186.242 - ||grad||^2 = 191.90208
 * Iteration #10 - Loss = 4181.760 - ||grad||^2 = 155.24151
 * Iteration #11 - Loss = 4178.588 - ||grad||^2 = 125.47734
 * Iteration #12 - Loss = 4176.350 - ||grad||^2 = 101.32337
 * Iteration #13 - Loss = 4174.782 - ||grad||^2 = 81.73722
 * Iteration #14 - Loss = 4173.691 - ||grad||^2 = 65.87097
 * Iteration #15 - Loss = 41

/Users/wangyuanzhang/anaconda3/envs/wsc/lib/python3.8/site-packages/pysurvival/utils/display.py:225: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


C-index of train: 0.7167; C-index of test: 0.6964
IBS: 0.17
Iteration 1 ... 

% Completion: 100%|**********************************************|Loss: 3361.84
% Completion: 100%|**********************************************|Loss: 4142.58
% Completion: 100%|**********************************************|Loss: 3600.61
% Completion: 100%|**********************************************|Loss: 3400.12
% Completion: 100%|**********************************************|Loss: 3737.45


...

Average c-index: 0.6954 Best so far
Iteration 2 ... Average c-index: 0.7224 Best so far
Iteration 3 ... Average c-index: 0.7348 Best so far
Iteration 4 ... Average c-index: 0.7380 Best so far
Iteration 5 ... Average c-index: 0.7329 
Iteration 6 ... Average c-index: 0.7390 Best so far
Iteration 7 ... Average c-index: 0.7405 Best so far
Iteration 8 ... Average c-index: 0.7272 
Iteration 9 ... Average c-index: 0.7339 
Iteration 10 ... Average c-index: 0.7342 
Iteration 11 ... Average c-index: 0.7376 
Iteration 12 ... Average c-index: 0.7329 
Iteration 13 ... Average c-index: 0.7242 
Iteration 14 ... Average c-index: 0.6915 
Iteration 15 ... Average c-index: 0.7108 
Iteration 16 ... Average c-index: 0.7333 
Iteration 17 ... Average c-index: 0.7211 
Iteration 18 ... Average c-index: 0.7222 
Iteration 19 ... Average c-index: 0.7304 
Iteration 20 ... Average c-index: 0.7270 
Iteration 21 ... Average c-index: 0.7365 
Iteration 22 ... Average c-index: 0.7311 
Iteration 23 ... Average c-index: 0

/Users/wangyuanzhang/anaconda3/envs/wsc/lib/python3.8/site-packages/pysurvival/utils/display.py:225: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


RSF - IBS: 0.16


/Users/wangyuanzhang/anaconda3/envs/wsc/lib/python3.8/site-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


Saving the model to disk as /Users/wangyuanzhang/Desktop/LHCC/python/RSF.zip
All models and results have been saved.
Feature importance analysis completed and saved.
